### 시계열 예측하기

In [12]:
import numpy as np
from tensorflow import keras
import tensorflow as tf

C:\Users\knuyh\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
def generate_time_series(batch_size, n_steps) : # 단변량
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # 사인 곡선1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # +사인 곡선2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # +잡읍
    return series[..., np.newaxis].astype(np.float32) # (배치크기, 타임스텝수, 1) 크기 반환

In [5]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [7]:
X_train.shape, y_train.shape

((7000, 50, 1), (7000, 1))

In [23]:
# 기준 성능 - 순진한 예측 : 각 시계열의 마지막 값 그대로 예측
y_pred = X_valid[:, -1] # (2000, 1)
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020684227

In [14]:
# 완전 연결 네트워크 이용 - 선형 예측
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [50, 1]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 9s 20ms/step - loss: 0.1056 - val_loss: 0.0453
Epoch 2/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0305 - val_loss: 0.0244
Epoch 3/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0177 - val_loss: 0.0151
Epoch 4/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0120 - val_loss: 0.0108
Epoch 5/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0093 - val_loss: 0.0086
Epoch 6/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0078 - val_loss: 0.0074
Epoch 7/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0069 - val_loss: 0.0064
Epoch 8/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0061 - val_loss: 0.0058
Epoch 9/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0056 - val_loss: 0.0053
Epoch 10/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0051 - val_loss

In [16]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 1s 7ms/step - loss: 0.0035


0.003547875676304102

In [17]:
# 간단한 RNN 구현하기
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])
# RNN은 어떤 길이의 타임 스텝도 처리할 수 있기에, 입력 시퀀스의 길이 지정할 필요 없다.

optimizer = keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 16s 47ms/step - loss: 0.0444 - val_loss: 0.0168
Epoch 2/20
219/219 [==============================] - 9s 41ms/step - loss: 0.0126 - val_loss: 0.0121
Epoch 3/20
219/219 [==============================] - 9s 39ms/step - loss: 0.0115 - val_loss: 0.0118
Epoch 4/20
219/219 [==============================] - 8s 37ms/step - loss: 0.0113 - val_loss: 0.0115
Epoch 5/20
219/219 [==============================] - 7s 30ms/step - loss: 0.0111 - val_loss: 0.0115
Epoch 6/20
219/219 [==============================] - 7s 30ms/step - loss: 0.0111 - val_loss: 0.0114
Epoch 7/20
219/219 [==============================] - 5s 23ms/step - loss: 0.0111 - val_loss: 0.0114
Epoch 8/20
219/219 [==============================] - 6s 25ms/step - loss: 0.0111 - val_loss: 0.0114
Epoch 9/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0111 - val_loss: 0.0114
Epoch 10/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0111 - val_l

In [18]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 1s 10ms/step - loss: 0.0114


0.011437698267400265

In [19]:
# 심층 RNN
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 14s 43ms/step - loss: 0.0311 - val_loss: 0.0078
Epoch 2/20
219/219 [==============================] - 10s 44ms/step - loss: 0.0060 - val_loss: 0.0050
Epoch 3/20
219/219 [==============================] - 9s 41ms/step - loss: 0.0050 - val_loss: 0.0049
Epoch 4/20
219/219 [==============================] - 8s 36ms/step - loss: 0.0043 - val_loss: 0.0042
Epoch 5/20
219/219 [==============================] - 8s 35ms/step - loss: 0.0039 - val_loss: 0.0042
Epoch 6/20
219/219 [==============================] - 7s 32ms/step - loss: 0.0036 - val_loss: 0.0037
Epoch 7/20
219/219 [==============================] - 7s 33ms/step - loss: 0.0037 - val_loss: 0.0036
Epoch 8/20
219/219 [==============================] - 7s 33ms/step - loss: 0.0036 - val_loss: 0.0036
Epoch 9/20
219/219 [==============================] - 8s 38ms/step - loss: 0.0035 - val_loss: 0.0039
Epoch 10/20
219/219 [==============================] - 9s 42ms/step - loss: 0.0035 - val_

In [20]:
model.evaluate(X_valid, y_valid) 

63/63 [==============================] - 1s 19ms/step - loss: 0.0037


0.0037151025608181953

In [22]:
# 기본적으로 tanh 활성화 함수 사용, 다른 활성화 함수 사용
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 8s 26ms/step - loss: 0.0136 - val_loss: 0.0048
Epoch 2/20
219/219 [==============================] - 5s 21ms/step - loss: 0.0043 - val_loss: 0.0039
Epoch 3/20
219/219 [==============================] - 5s 22ms/step - loss: 0.0038 - val_loss: 0.0039
Epoch 4/20
219/219 [==============================] - 5s 24ms/step - loss: 0.0036 - val_loss: 0.0034
Epoch 5/20
219/219 [==============================] - 6s 28ms/step - loss: 0.0035 - val_loss: 0.0034
Epoch 6/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0034 - val_loss: 0.0033
Epoch 7/20
219/219 [==============================] - 8s 37ms/step - loss: 0.0033 - val_loss: 0.0037
Epoch 8/20
219/219 [==============================] - 8s 37ms/step - loss: 0.0032 - val_loss: 0.0031
Epoch 9/20
219/219 [==============================] - 7s 33ms/step - loss: 0.0032 - val_loss: 0.0031
Epoch 10/20
219/219 [==============================] - 10s 44ms/step - loss: 0.0032 - val_l

In [24]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 1s 10ms/step - loss: 0.0029


0.0029373790603131056

MSE로 보는  
성능 : 순진한 예측 < 간단한 RNN < 선형 모델 < 심층 RNN < 출력층 Dense

In [26]:
# 여러 타임 스텝 앞을 예측하기
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new

for step_ahead in range(10) :
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

1/1 [==============================] - 0s 43ms/step


In [29]:
Y_pred = X[:, n_steps:]
Y_pred.shape

(1, 10, 1)